In [1]:
import fire
from visdom import Visdom
import pickle
import sys, os
import umap, numba
from sklearn.preprocessing import LabelEncoder
import numpy as np
import os,glob, pandas as pd
from sklearn.metrics import f1_score
import copy
from collections import Counter
import torch
import torch.nn.functional as F
from torch import nn
from torch_geometric.nn import GCNConv, GATConv, DeepGraphInfomax, SAGEConv
from torch_geometric.nn import DenseGraphConv
from torch_geometric.utils import to_dense_batch, to_dense_adj, dense_to_sparse
from torch_geometric.nn import GINEConv
from torch_geometric.utils import dropout_adj
from torch_geometric.nn import APPNP
from torch_cluster import knn_graph
from torch_geometric.data import Data 
from torch_geometric.utils import train_test_split_edges
from torch_geometric.utils.convert import to_networkx
from torch_geometric.data import InMemoryDataset,DataLoader
from sklearn.utils.class_weight import compute_class_weight

2022-12-22 17:53:06.560074: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
EPS = 1e-15

class GCNNet(torch.nn.Module):
    def __init__(self, inp_dim, out_dim, hidden_topology=[32,64,128,128], p=0.5, p2=0.1, drop_each=True):
        super(GCNNet, self).__init__()
        self.out_dim=out_dim
        self.convs = nn.ModuleList([GATConv(inp_dim, hidden_topology[0])]+[GATConv(hidden_topology[i],hidden_topology[i+1]) for i in range(len(hidden_topology[:-1]))])
        self.drop_edge = lambda edge_index: dropout_adj(edge_index,p=p2)[0]
        self.dropout = nn.Dropout(p)
        self.fc = nn.Linear(hidden_topology[-1], out_dim)
        self.drop_each=drop_each

    def forward(self, x, edge_index, edge_attr=None):
        for conv in self.convs:
            if self.drop_each and self.training: edge_index=self.drop_edge(edge_index)
            x=conv(x, edge_index, edge_attr)
            x = F.relu(x)
        z=x
        if self.training:
            z = self.dropout(z)
        x = self.fc(z)
        return x,z
    
class GCNFeatures(torch.nn.Module):
    def __init__(self, gcn, bayes=False):
        super(GCNFeatures, self).__init__()
        self.gcn=gcn
        self.drop_each=bayes
    
    def forward(self, x, edge_index, edge_attr=None):
        for conv in self.gcn.convs:
            if self.drop_each: edge_index=self.gcn.drop_edge(edge_index)
            x = F.relu(conv(x, edge_index, edge_attr))
        if self.drop_each:
            x = self.gcn.dropout(x)
        y = F.softmax(self.gcn.fc(x))
        return x,y

def fit_model(graph_data='',
                use_weights=False,
                use_model=None,
                n_batches_backward=1,
                f1_metric='weighted',
                n_epochs=1500,
                out_dir="/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/gnn/",
                lr=1e-2,
                eta_min=1e-4,
                T_max=20,
                wd=0,
                hidden_topology=[32,64,128,128],
                p=0.5,
                p2=0.3,
                burnin=400,
                warmup=100,
                gpu_id=0,
                batch_size=1
                ):
    print(gpu_id); torch.cuda.set_device(gpu_id)
    datasets=pickle.load(open(graph_data,'rb'))
    train_dataset = [dataset for key in datasets for dataset in datasets[key]['train']]
    val_dataset = [dataset for key in datasets for dataset in datasets[key]['val']]
    print(len(train_dataset), 'training graphs,', len(val_dataset), 'validation graphs')
    
    print("training graph sizes:", end='')
    for x in train_dataset:
        print(x.x.shape[0], end=", ")
    print("\nValidation graph sizes:", end=" ")
    for x in val_dataset:
        print(x.x.shape[0], end=", ")
 
    y_train=np.hstack([graph.y.numpy() for graph in train_dataset])

    if use_weights: 
        weights=compute_class_weight('balanced',classes=np.unique(y_train),y=y_train)
    else: 
        weights=None 
       
    # load model
    model=GCNNet(train_dataset[0].x.shape[1],len(np.unique(y_train)),hidden_topology=hidden_topology,p=p,p2=p2)
    model=model.cuda()

    # load optimizer
    optimizer = torch.optim.Adam(model.parameters(),lr=lr,weight_decay=wd)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max, eta_min=eta_min, last_epoch=-1)
    criterion=nn.CrossEntropyLoss(weight=torch.tensor(weights).float() if use_weights else None)
    criterion=criterion.cuda()

    # initialize val saving
    save_mod=False
    past_performance=[0]

    # dataloaders
    dataloaders={}

    dataloaders['train']=DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
    dataloaders['val']=DataLoader(val_dataset,shuffle=True)
    dataloaders['warmup']=DataLoader(train_dataset,shuffle=False)
    train_loader=dataloaders['warmup']

    n_total_batches=0
    train_val_f1=[]
    for epoch in range(n_epochs):
        Y,Y_Pred=[],[]
        for i,data in enumerate(train_loader):
            n_total_batches+=1
            model.train(True)
            x=data.x.cuda()
            edge_index=data.edge_index.cuda()
            y=data.y.cuda()
            y_out=model(x,edge_index)
            loss = criterion(y_out, y) / n_batches_backward
            loss.backward()
            if n_total_batches%n_batches_backward==0 or (i==len(train_loader.dataset)-1):
                optimizer.step()
                optimizer.zero_grad()
            Y_Pred.append(F.softmax(y_out, dim=1).argmax(1).detach().cpu().numpy().flatten())
            Y.append(y.detach().cpu().numpy().flatten())
            del x, edge_index, loss, y_out
            if epoch <=warmup:
                break 
        if epoch == warmup:
            train_loader=dataloaders['train']
        if epoch>=burnin:
            save_mod=True
        train_f1=f1_score(np.hstack(Y),np.hstack(Y_Pred),average=f1_metric)
        scheduler.step()
        Y,Y_Pred=[],[]
        for i,data in enumerate(dataloaders['val']):
            model.train(False)
            x=data.x.cuda()
            edge_index=data.edge_index.cuda()
            y=data.y.cuda()
            y_out=model(x,edge_index)
            loss = criterion(y_out, y) 
            y_prob=F.softmax(y_out, dim=1).detach().cpu().numpy()
            y_pred=y_prob.argmax(1).flatten()
            y_true=y.detach().cpu().numpy().flatten()
            Y_Pred.append(y_pred)
            Y.append(y_true)
            #if vis_every and epoch%vis_every==0 and not i:
               # vis.scatter(data.pos.numpy(),opts=dict(markercolor=(y_pred*255).astype(int),webgl=False,markerborderwidth=0,markersize=5),win="pred")
               # vis.scatter(data.pos.numpy(),opts=dict(markercolor=y_true*255,webgl=False,markerborderwidth=0,markersize=5),win="true")
            del x, edge_index, loss, y_out
        val_f1=f1_score(np.hstack(Y),np.hstack(Y_Pred),average=f1_metric)
        if save_mod and val_f1>=max(past_performance):
            best_model_dict=copy.deepcopy(model.state_dict())
            past_performance.append(val_f1)
        print(epoch,train_f1,val_f1,flush=True)
        train_val_f1.append((train_f1,val_f1))

    model.load_state_dict(best_model_dict)
    torch.save(model.state_dict(),os.path.join(out_dir,f"model.pth"))
    torch.save(train_val_f1,os.path.join(out_dir,f"f1.log.pth"))
    return Y, Y_Pred

In [ ]:
y, y_pred = fit_model("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/graph_dataset.pkl", 
                      use_weights=True, 
                      lr=1e-4, 
                      out_dir='/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/gnn/', 
                      n_epochs=1000, 
                      gpu_id=0, 
                      burnin=0)


0
234 training graphs, 90 validation graphs
training graph sizes:20589, 5563, 5101, 5719, 5649, 6098, 6239, 4220, 4557, 4607, 817, 795, 703, 832, 668, 884, 583, 902, 815, 878, 844, 826, 871, 842, 2626, 1742, 2955, 2561, 2825, 7126, 6957, 5176, 4841, 5075, 4659, 5137, 5193, 10733, 10457, 10168, 10205, 7420, 6622, 7200, 6892, 6043, 8925, 7523, 8040, 18384, 3285, 3282, 2217, 2806, 817, 795, 703, 832, 668, 884, 583, 902, 815, 878, 844, 826, 871, 842, 6056, 217, 22341, 8943, 23840, 2319, 2536, 2706, 2642, 2598, 2578, 5059, 4642, 5019, 5072, 2608, 532, 2529, 2207, 2774, 2110, 256, 647, 1926, 256, 1968, 2800, 369, 358, 2152, 2573, 5921, 2435, 6168, 6601, 6353, 6222, 5494, 5831, 4710, 5083, 1686, 1663, 1787, 1805, 1665, 3270, 3424, 3966, 3803, 3460, 3076, 3317, 3746, 3797, 3216, 2713, 2708, 2900, 2921, 2752, 2578, 2365, 2314, 2572, 2198, 2440, 2460, 2678, 2647, 2462, 4220, 4557, 4607, 13521, 15681, 18198, 459, 8477, 2248, 11270, 12102, 7910, 2793, 2950, 3738, 3631, 2626, 1742, 2955, 2561, 2825

/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


0 0.7576670517340872 0.6567232646383594
1 0.8925171231906389 0.6827593169576053
2 0.9512308352798088 0.7251630603274609
3 0.9673065460700395 0.7359452608350822
4 0.9745752404400914 0.7599332948809647
5 0.977137020256 0.7833815680988803
6 0.9783759051111769 0.786171318040969
7 0.9779372361278236 0.7871418053807854
8 0.9791518868436854 0.784315243681778
9 0.9779680203448804 0.7774041537835195
10 0.9760511003583089 0.7713587754330876
11 0.9747361686781612 0.7697116096982307
12 0.9706079751963161 0.7654883510923532
13 0.9670623111607559 0.7639590032773195
14 0.9655244419586676 0.767050485891488
15 0.9637469744988574 0.7676360094209151
16 0.9646395830392398 0.7662928870647098
17 0.9663632881747346 0.7665612983980747
18 0.9680590687880833 0.7676409274318248
19 0.9692876817253276 0.7762777962203208
20 0.9708484400608575 0.7815885963530007
21 0.972419910056201 0.7818423464566379
22 0.9743585572571725 0.7796819754603438
23 0.9754153241272533 0.7778651185414359
24 0.974906027571371 0.78252186538

In [13]:
y, y_pred= fit_model("/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/graph_dataset/graph_dataset.pkl",
                    use_weights=True,
                    lr=1e-4,
                    out_dir='/dartfs/rc/nosnapshots/V/VaickusL-nb/EDIT_Students/users/Sophie_Chen/gnn/',
                    n_epochs=1500,
                    gpu_id=0,
                    burnin=0)

0
234 training graphs, 90 validation graphs
training graph sizes:20589, 5563, 5101, 5719, 5649, 6098, 6239, 4220, 4557, 4607, 817, 795, 703, 832, 668, 884, 583, 902, 815, 878, 844, 826, 871, 842, 2626, 1742, 2955, 2561, 2825, 7126, 6957, 5176, 4841, 5075, 4659, 5137, 5193, 10733, 10457, 10168, 10205, 7420, 6622, 7200, 6892, 6043, 8925, 7523, 8040, 18384, 3285, 3282, 2217, 2806, 817, 795, 703, 832, 668, 884, 583, 902, 815, 878, 844, 826, 871, 842, 6056, 217, 22341, 8943, 23840, 2319, 2536, 2706, 2642, 2598, 2578, 5059, 4642, 5019, 5072, 2608, 532, 2529, 2207, 2774, 2110, 256, 647, 1926, 256, 1968, 2800, 369, 358, 2152, 2573, 5921, 2435, 6168, 6601, 6353, 6222, 5494, 5831, 4710, 5083, 1686, 1663, 1787, 1805, 1665, 3270, 3424, 3966, 3803, 3460, 3076, 3317, 3746, 3797, 3216, 2713, 2708, 2900, 2921, 2752, 2578, 2365, 2314, 2572, 2198, 2440, 2460, 2678, 2647, 2462, 4220, 4557, 4607, 13521, 15681, 18198, 459, 8477, 2248, 11270, 12102, 7910, 2793, 2950, 3738, 3631, 2626, 1742, 2955, 2561, 2825

/dartfs-hpc/rc/home/3/f006n33/anaconda3/envs/hiss/lib/python3.8/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/scratch/ipykernel_116240/1767626643.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  Y_Pred.append(F.softmax(y_out).argmax(1).detach().cpu().numpy().flatten())
/scratch/ipykernel_116240/1767626643.py:133: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_prob=F.softmax(y_out).detach().cpu().numpy()


0 0.574741640886651 0.6261320531429796


/scratch/ipykernel_116240/1767626643.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  Y_Pred.append(F.softmax(y_out).argmax(1).detach().cpu().numpy().flatten())
/scratch/ipykernel_116240/1767626643.py:133: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_prob=F.softmax(y_out).detach().cpu().numpy()


1 0.7950344363929887 0.65062912760669


...

142 0.7325499902515851 0.7922869683027197


/scratch/ipykernel_116240/1767626643.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  Y_Pred.append(F.softmax(y_out).argmax(1).detach().cpu().numpy().flatten())
/scratch/ipykernel_116240/1767626643.py:133: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  y_prob=F.softmax(y_out).detach().cpu().numpy()


143 0.7325499902515851 0.7922869683027197


/scratch/ipykernel_116240/1767626643.py:114: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  Y_Pred.append(F.softmax(y_out).argmax(1).detach().cpu().numpy().flatten())


KeyboardInterrupt: 